# <center>**Práctica 3 - Parte 2**
## <center>**Comparación estadística de energía de señales EEG**
#### Maria Alejandra Escobar - C.C 1004425012
#### Juan Manuel Martínez - C.C 1007739919

#Punto 0
Carga  y exploración de datos

In [2]:
# Importar librerías
import numpy as np;
import matplotlib.pyplot as plt
from csv import reader as reader_csv
import scipy.signal as signal
import scipy.io as sio
import glob
import pandas as pd
import scipy.stats as stats

In [3]:
# Conectar colab con drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
datos_control = glob.glob('/content/drive/MyDrive/Colab Notebooks/Practica_3/control/*.mat')
datos_parkinson = glob.glob('/content/drive/MyDrive/Colab Notebooks/Practica_3/parkinson/*.mat')

In [5]:
print(datos_control)

['/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C005_EP_reposo_Repetido.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C002_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C001R_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C004_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C006_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C012_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C007_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C010_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C011_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C020_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C021_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C023_EP_reposo.mat', '/content/drive/MyDrive/Colab

In [6]:
# Dimensiones de los datos
dimension_control = [sio.loadmat(x)['data'].shape for x in datos_control]
dimension_parkinson = [sio.loadmat(x)['data'].shape for x in datos_parkinson]

print(dimension_control)
print(dimension_parkinson)
print(len(dimension_control))
print(len(dimension_parkinson))

[(8, 2000, 149), (8, 2000, 176), (8, 2000, 180), (8, 2000, 146), (8, 2000, 168), (8, 2000, 172), (8, 2000, 173), (8, 2000, 136), (8, 2000, 173), (8, 2000, 166), (8, 2000, 180), (8, 2000, 206), (8, 2000, 157), (8, 2000, 186), (8, 2000, 175), (8, 2000, 179), (8, 2000, 140), (8, 2000, 187), (8, 2000, 144), (8, 2000, 173), (8, 2000, 167), (8, 2000, 189), (8, 2000, 181), (8, 2000, 142), (8, 2000, 147), (8, 2000, 169), (8, 2000, 161), (8, 2000, 160), (8, 2000, 258), (8, 2000, 203), (8, 2000, 154), (8, 2000, 154), (8, 2000, 163), (8, 2000, 178), (8, 2000, 184), (8, 2000, 164)]
[(8, 2000, 138), (8, 2000, 143), (8, 2000, 176), (8, 2000, 140), (8, 2000, 150), (8, 2000, 174), (8, 2000, 143), (8, 2000, 147), (8, 2000, 149), (8, 2000, 153), (8, 2000, 140), (8, 2000, 154), (8, 2000, 154), (8, 2000, 170), (8, 2000, 142), (8, 2000, 167), (8, 2000, 174), (8, 2000, 198), (8, 2000, 171), (8, 2000, 162), (8, 2000, 157), (8, 2000, 164), (8, 2000, 159)]
36
23


In [7]:
print(type(datos_control))
print(datos_control)

<class 'list'>
['/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C005_EP_reposo_Repetido.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C002_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C001R_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C004_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C006_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C012_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C007_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C010_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C011_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C020_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C021_EP_reposo.mat', '/content/drive/MyDrive/Colab Notebooks/Practica_3/control/C023_EP_reposo.mat', '/content/driv

#Punto 1
Implemente una función que reciba una señal de múltiples canales y épocas y calcule la Energía de promedio de cada canal.

In [8]:
def promedio_energia(data):
  resultado=np.zeros(data.shape[0]) #Se crea una matriz de ceros para almacenar la energia promedio de cada canal
  for c in range (data.shape[0]): # Recorre cada canal
    energia_epoca = 0
    for e in range (data.shape[2]): # Recorre epocas dentro de cana canal
      energia_datos = np.sum(np.square(data[c,:,e])) # Se saca promedio de los  datos de una epoca
      num_datos = data.shape[1]
      energiapromedio_datos = energia_datos/num_datos
      energia_epoca = energia_epoca + energiapromedio_datos #primer valor será 0, se suman las energias de cada epoca
    num_epocas=data.shape[2]
    energia_canal = energia_epoca/num_epocas
    resultado[c] = energia_canal # Se almacena en la matriz las energias de cada canal por sujeto en columnas

  return resultado

In [9]:
for datos in datos_control:
  control= sio.loadmat(datos)
  data = control['data']
  energia = promedio_energia(data)
  print(energia)

[17.65565085 17.45834301 19.40021451 17.71351556 17.95273643 53.29906408
 53.44278798 56.26037532]
[ 7.98320143  8.80890512 10.40246856  9.82720001  8.33949103 46.9470245
 33.43124814 37.84256294]
[10.73282518 10.49295396 11.38007479  9.25282014 14.86508151 12.62207904
 11.39066379 12.32929976]
[ 7.07433666  9.14199983 14.37496607  7.13536346 14.39372299  7.33070887
  7.97007705  9.74994933]
[ 9.25541499  9.86924469 10.45589637 10.9141272  11.67599632 26.54302988
 18.74798624 21.53354775]
[ 4.56651815  4.60707751  5.81320591  5.40481081  5.23393997 23.16843677
 22.84790088 25.04318082]
[ 6.59005466  6.96260891  8.10949711  6.16244183  7.03015033 12.88351243
 10.96787981 11.41389065]
[ 5.59877729  5.4741844   6.36850233  5.37258096  5.16482121 10.73080292
  7.74660639 13.70718762]
[14.27556203 13.10241963  8.69199948  8.62230297 13.10318621 41.6853092
 25.56055286 33.92617413]
[ 2.45753139  3.05292497  2.58377617  2.06885893  2.05199764 10.17979317
  8.79948129 11.14611523]
[12.90379934

#Punto 2
Calcule la energía de cada canal promediada por épocas para cada sujeto, esto para ambos grupos poblacionales. Guarde esta información en un DataFrame de columnas ‘canal’ y filas ‘#sujeto’ con los valores de energía calculados, un DataFrame para cada grupo poblacional.

In [10]:
s_control = np.zeros((len(datos_control),8)) # Se crea un matriz de ceros con 36 filas (numero de sujetos de contol) y 8 columnas correspondientes a los canales

for datos in range(len((datos_control))): # Itera sobre los sujetos
  data = sio.loadmat(datos_control[datos])
  info = data['data']
  i = promedio_energia(info)
  s_control[datos] = i
df_control = pd.DataFrame ({'Canal 1':pd.Series(s_control[:,0]),'Canal 2':pd.Series(s_control[:,1]),'Canal 3':pd.Series(s_control[:,2]),'Canal 4':pd.Series(s_control[:,3]),'Canal 4':pd.Series(s_control[:,3]),'Canal 5':pd.Series(s_control[:,4]),'Canal 6':pd.Series(s_control[:,5]),'Canal 7':pd.Series(s_control[:,6]),'Canal 8':pd.Series(s_control[:,7])})
df_control = df_control.rename(index={0: 'Sujeto 1', 1: 'Sujeto 2', 2: 'Sujeto 3', 3: 'Sujeto 4', 4: 'Sujeto 5', 5: 'Sujeto 6', 6: 'Sujeto 7', 7: 'Sujeto 8', 8: 'Sujeto 9', 9: 'Sujeto 10', 10: 'Sujeto 11', 11: 'Sujeto 12', 12: 'Sujeto 13', 13: 'Sujeto 14', 14: 'Sujeto 15', 15: 'Sujeto 16', 16: 'Sujeto 17', 17: 'Sujeto 18', 18: 'Sujeto 19', 19: 'Sujeto 20', 20: 'Sujeto 21', 21: 'Sujeto 22', 22: 'Sujeto 23', 23: 'Sujeto 24', 24: 'Sujeto 25', 25: 'Sujeto 26', 26: 'Sujeto 27',27: 'Sujeto 28',28: 'Sujeto 29',29: 'Sujeto 30',30: 'Sujeto 31',31: 'Sujeto 32',32: 'Sujeto 33',33: 'Sujeto 34',34: 'Sujeto 35',35: 'Sujeto 36'})
print(df_control)

             Canal 1    Canal 2    Canal 3    Canal 4    Canal 5     Canal 6  \
Sujeto 1   17.655651  17.458343  19.400215  17.713516  17.952736   53.299064   
Sujeto 2    7.983201   8.808905  10.402469   9.827200   8.339491   46.947025   
Sujeto 3   10.732825  10.492954  11.380075   9.252820  14.865082   12.622079   
Sujeto 4    7.074337   9.142000  14.374966   7.135363  14.393723    7.330709   
Sujeto 5    9.255415   9.869245  10.455896  10.914127  11.675996   26.543030   
Sujeto 6    4.566518   4.607078   5.813206   5.404811   5.233940   23.168437   
Sujeto 7    6.590055   6.962609   8.109497   6.162442   7.030150   12.883512   
Sujeto 8    5.598777   5.474184   6.368502   5.372581   5.164821   10.730803   
Sujeto 9   14.275562  13.102420   8.691999   8.622303  13.103186   41.685309   
Sujeto 10   2.457531   3.052925   2.583776   2.068859   2.051998   10.179793   
Sujeto 11  12.903799  15.574167  14.320193  10.512889   9.125902   74.112767   
Sujeto 12   7.768116  13.585777  15.6475

In [11]:
s_parkinson = np.zeros((len(datos_parkinson),8))

for d in range(len(datos_parkinson)):
  data = sio.loadmat(datos_parkinson[d])
  datos = data['data']
  j = promedio_energia(datos)
  s_parkinson[d] = j

df_parkinson = pd.DataFrame ({'Canal 1':pd.Series(s_parkinson[:,0]),'Canal 2':pd.Series(s_parkinson[:,1]),'Canal 3':pd.Series(s_parkinson[:,2]),'Canal 4':pd.Series(s_parkinson[:,3]),'Canal 4':pd.Series(s_parkinson[:,3]),'Canal 5':pd.Series(s_parkinson[:,4]),'Canal 6':pd.Series(s_parkinson[:,5]),'Canal 7':pd.Series(s_parkinson[:,6]),'Canal 8':pd.Series(s_parkinson[:,7])})
df_parkinson = df_parkinson.rename(index={0: 'Sujeto 1', 1: 'Sujeto 2', 2: 'Sujeto 3', 3: 'Sujeto 4', 4: 'Sujeto 5', 5: 'Sujeto 6', 6: 'Sujeto 7', 7: 'Sujeto 8', 8: 'Sujeto 9', 9: 'Sujeto 10', 10: 'Sujeto 11', 11: 'Sujeto 12', 12: 'Sujeto 13', 13: 'Sujeto 14', 14: 'Sujeto 15', 15: 'Sujeto 16', 16: 'Sujeto 17', 17: 'Sujeto 18', 18: 'Sujeto 19', 19: 'Sujeto 20', 20: 'Sujeto 21', 21: 'Sujeto 22', 22: 'Sujeto 23'})
print(df_parkinson)

             Canal 1    Canal 2    Canal 3    Canal 4    Canal 5     Canal 6  \
Sujeto 1    8.997830   6.000801   6.143172   7.392954   8.529217   31.991725   
Sujeto 2    6.219122   5.630588   5.409817   4.744892   6.045530   11.399107   
Sujeto 3    4.790905   7.004286   4.794615   4.687043   4.077471   14.485424   
Sujeto 4   11.723026  13.045671  18.320266  12.830111  15.665275   46.848064   
Sujeto 5    6.078615   6.699329   8.834439   7.420552   5.648871   19.350824   
Sujeto 6   19.046051  21.787690  20.989997  20.857644  23.256869  125.824697   
Sujeto 7    8.101208   6.562124   6.994337   6.376014   7.892362   25.365117   
Sujeto 8    5.346474   5.420594   6.077195  12.080843   7.394587   21.651413   
Sujeto 9   11.829369  11.995128  15.316873  11.444447   9.966158   32.580716   
Sujeto 10  11.871163  11.035004  12.270158  10.901968  11.297170   64.157132   
Sujeto 11  24.287259  25.903265  36.585976  29.853850  28.276088  143.552881   
Sujeto 12   8.849514  13.321640  16.2344

# Punto 3
Determine si existe diferencia estadística entre canales de cada grupo de sujetos a través de una prueba t. Compruebe los supuestos necesarios para realizar una prueba t, esto es: Normalidad de la variable, independencia (se asume que los grupos son independientes), y homocedasticidad (use una prueba de Levene), finalmente realice la prueba t para determinar si existen diferencias entre los canales entre grupos de sujetos. De no cumplirse los requisitos, realice entonces un análisis no paramétrico (prueba U de Mann-Whitney). Este numeral tiene como objetivo identificar los canales que entregan información diferencial entre pacientes Sanos y con enfermedad de Parkinson.

####Se aplica prueba Shapiro porque n<50 , para evaluar inicialmente la ditribución de los datos y corroborar supuestos para evaluar prueba t. Se escoge un **alpha= 0.05**

In [12]:
from scipy.stats import ttest_ind,shapiro, levene
from sklearn.preprocessing import power_transform

In [23]:
PC=[] # se crea un vector vacio para almacenar el valor de p
i=0
for i in range(s_control.shape[1]):
  n_control = s_control[:,i] # se extrae el valor de la energia promedio de cada uno de los canales de todos los sujetos control
  stat, p = shapiro(n_control) # se realiza la prueba shapiro
  PC.append(p) # se agregan los valores de p al vector
  i = i+1 # se crea un contador para entrar a diferentes canales
print('Ho: Los datos de sujetos de control provienen de una distribución normal.')
print('Ha: Los datos de sujetos de control no provienen de una distribución normal.')
print('Resultado de prueba Shapiro para datos de control:')
print(PC)

Ho: Los datos de sujetos de control provienen de una distribución normal.
Ha: Los datos de sujetos de control no provienen de una distribución normal.
Resultado de prueba Shapiro para datos de control:
[0.006252463907003403, 0.0039543164893984795, 0.008902405388653278, 0.00019698070536833256, 0.0007995034102350473, 6.077864782128017e-06, 4.219014954287559e-06, 5.307714445734746e-07]


In [22]:
PP=[]
i=0
for i in range(s_parkinson.shape[1]):
  n_parkinson = s_parkinson[:,i] # se extrae el valor de la energia promedio de cada uno de los canales de todos los sujetos parkinson
  stat, p = shapiro(n_parkinson)
  PP.append(p)
  i = i+1

print('Ho: Los datos de sujetos de parkinson provienen de una distribución normal.')
print('Ha: Los datos de sujetos de parkinson no provienen de una distribución normal.')
print('Resultado de prueba Shapiro para datos de parkinson:')
print(PP)

Ho: Los datos de sujetos de parkinson provienen de una distribución normal.
Ha: Los datos de sujetos de parkinson no provienen de una distribución normal.
Resultado de prueba Shapiro para datos de parkinson:
[0.014391767792403698, 0.00452211732044816, 0.0009095783461816609, 0.00031213118927553296, 0.005323005840182304, 9.81468656391371e-06, 1.670212805038318e-05, 8.983333827927709e-06]


Los resultados de la prueba shapiro arrojan valores por debajo del aplha escogido (0.05) de esta manera rechazandose la hipótesis nula. Esto nos permite decir que la distribución tanto de los datos de control como de parkinson no siguen una distribución normal.

Sin embargo se realizará la prueba de Levene de homocedasticidad para comprobar todos los supuestos. Se asume nuevamente el valor de **alpha=0.05**

In [21]:
#Se realiza la prueba de homocedasticidad para cada sujeto de datos de control y parkinson por canal
p_homo = [] #Lista vacía para agregar los valor de P de la prueba Levene de cada canal

#Canal1
suj_control = s_control[:,0]
suj_parkinson = s_parkinson[:,0]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal2
suj_control = s_control[:,1]
suj_parkinson = s_parkinson[:,1]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal3
suj_control = s_control[:,2]
suj_parkinson = s_parkinson[:,2]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal4
suj_control = s_control[:,3]
suj_parkinson = s_parkinson[:,3]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal5
suj_control = s_control[:,4]
suj_parkinson = s_parkinson[:,4]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal6
suj_control = s_control[:,5]
suj_parkinson = s_parkinson[:,5]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal7
suj_control = s_control[:,6]
suj_parkinson = s_parkinson[:,6]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)
#Canal8
suj_control = s_control[:,7]
suj_parkinson = s_parkinson[:,7]
estadistica, p = levene(suj_control, suj_parkinson)
p_homo.append(p)

print('Ho: Las varianzas entre los grupos son homogéneas.')
print('Ha: Las varianzas entre los grupos no son homogéneas.')
print(p_homo)

Ho: Las varianzas entre los grupos son homogéneas.
Ha: Las varianzas entre los grupos no son homogéneas.
[0.8848020019348577, 0.9590926426745954, 0.9776337925998679, 0.7592528471707682, 0.9045548264900758, 0.4719758936966435, 0.6178759981277988, 0.5940765419878595]


Los resultados de la prueba de homocedasticidad de sujetos de control y parkinson por canal, arroja valor de P mayores al alpha escogido, por lo cual no se rechaza la hipótesis nula y se puede decir que los datos cumplen el supuesto de que los valores de sus varianzas no están muy alejados entre sí y no tienen diferencias significativas.

Finalmente, se asume independencia de los datos y aunque se cumpla también con el suspuesto de homocedasticidad, se realizará una prueba no paramétrica Mann Whitney porque los datos no cumplen con una distribución normal.

In [18]:
import scipy.stats as stats

In [20]:
#Se realiza la prueba de Mann Whitney para cada sujeto de datos de control y parkinson por canal
p_mann = [] #Lista vacía para agregar los valor de P de la prueba Mann Whitney de cada canal

#Canal1
suj_control = s_control[:,0]
suj_parkinson = s_parkinson[:,0]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal2
suj_control = s_control[:,1]
suj_parkinson = s_parkinson[:,1]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal3
suj_control = s_control[:,2]
suj_parkinson = s_parkinson[:,2]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal4
suj_control = s_control[:,3]
suj_parkinson = s_parkinson[:,3]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal5
suj_control = s_control[:,4]
suj_parkinson = s_parkinson[:,4]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal6
suj_control = s_control[:,5]
suj_parkinson = s_parkinson[:,5]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal7
suj_control = s_control[:,6]
suj_parkinson = s_parkinson[:,6]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)
#Canal8
suj_control = s_control[:,7]
suj_parkinson = s_parkinson[:,7]
estadistica, p = stats.mannwhitneyu(suj_control, suj_parkinson)
p_mann.append(p)

print('Ho = Las medianas de los ciclos son homogeneas.')
print('Ha = Las medianas de los ciclos no son homogeneas.')
print(p_mann)

Ho = Las medianas de los ciclos son homogeneas.
Ha = Las medianas de los ciclos no son homogeneas.
[0.4057005715724912, 0.5705282028889593, 0.46037288907343565, 0.23446184956559823, 0.5600175059745567, 0.2800747649284743, 0.11830081572892007, 0.15054429358714408]


Una vez realizada la prueba U de Mann-Whitney se obtuvieron valores de mayores a 0.05 por lo cual no se rechaza la hipótesis nula. Se puede demostrar, con las hipótesis planteadas, que la energía de la señal en los diferentes canales entre los grupos control y Parkinson no presentan diferencias significativas, de lo cual se puede concluir que la energía de la señal no es un factor determinante, en el que se tenía que la energía promedio de los canales del EEG en pacientes con Parkinson son diferentes a la energía promedio de la señal EEG en pacientes sanos.

# Conclusiones
Los resultados obtenidos a partir de las pruebas estadisticas realizadas no proveen una perspectiva completa de las diferencias entre las poblaciones evaluadas, lo que da lugar a inferir que la energía promedio de canales no es un parametro que entregue información significativa.  En el análisis de datos realizado en el punto 3 con el objetivo de identificar canales o características informativas diferenciales entre pacientes sanos y pacientes con enfermedad de Parkinson, no se encontraron diferencias estadísticamente significativas. Esto puede significar que en los datos y en las variables evaluadas en este estudio, no hay características distintivas que permitan una clara distinción entre pacientes de control y con parkinson con la energía promedio de la señal.